In [1]:
import matplotlib
from matplotlib import pylab
%config InlineBackend.figure_format = 'retina'
matplotlib.rcParams['figure.dpi'] = 80
params = {'legend.fontsize': 'xx-large',
          'figure.figsize': (6, 4),
         'axes.labelsize': 'xx-large',
         'axes.titlesize': 'xx-large',
         'xtick.labelsize': 'x-large',
         'ytick.labelsize': 'x-large',
         'figure.facecolor': 'w'}
pylab.rcParams.update(params)
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

import h5py
import numpy as np
import sys

sys.path.insert(1, '../code')
from featurize_and_fit import Featurizer, Fitter
import plotter
import scalars

%load_ext autoreload
%autoreload 2

In [2]:
save_plots = False
show_intermediate_plots = False
check_cond = False
plot_dir = '../plots/plots_2022-03-03_catalog'
if save_plots:
    !mkdir -p $plot_dir

# r_edges (TNG50-4: L=35 Mpc/h, N_part=270^3, mean interparticle spacing = 35/270 = 0.13 Mpc/h = 130 kpc/h)
# for scalars, only 0 or 2 x orders 
n_rbins = 1
m_order_max = 3
x_order_max = 2
v_order_max = 0
include_eigenvalues = False
include_eigenvectors = False

catalog_feature_names = ['Group_M_Crit200'
                        ]

#y_scalar_feature_name = 'mass_hydro_halo_star'
y_scalar_feature_name='mass_hydro_subhalo_star'

r_units = 'r200'
rms_x = True
log_x = False
log_y = False

#tag = '_unc0.1Mfloor2e8'
#tag = '_logxlogy'
#tag = '_rmsfalse'
#tag = '_velocities'
tag = ''
if include_eigenvalues:
    tag += '_eigvals'

l_arr, p_arr = scalars.get_needed_vec_orders_scalars(x_order_max, v_order_max)

In [3]:
mass_multiplier = 1e10

In [4]:
base_dir = '/scratch/ksf293/equivariant-cosmology/data'
sim_name = 'TNG100-1'
snap_num_str = '099' # z = 0

In [5]:
featurizer = Featurizer()
featurizer.read_simulations(base_dir, sim_name, snap_num_str)
featurizer.match_twins()
featurizer.select_halos()
featurizer.add_info_to_halo_dicts()
featurizer.set_y_labels(y_scalar_feature_name=y_scalar_feature_name)

In [6]:
featurizer.get_catalog_features(catalog_feature_names)

NameError: name 'snap_num_str' is not defined

In [ ]:
print(featurizer.x_catalog_features)

In [9]:
featurizer.tng_path_dark

'/scratch/ksf293/equivariant-cosmology/dataTNG100-1-Dark'

In [11]:
f = h5py.File(f'/scratch/ksf293/equivariant-cosmology/data/TNG100-1-Dark/postprocessing/halo_structure_{snap_num_str}.hdf5', 'r')

In [12]:
print(f)
print(f.keys())

<HDF5 file "halo_structure_099.hdf5" (mode r+)>
<KeysViewHDF5 ['E_s', 'GroupFlag', 'Header', 'M200c', 'M_acc_dyn', 'Mean_vel', 'R0p9', 'a_form', 'c200c', 'f_mass_Cen', 'q', 'q_vel', 's', 's_vel', 'sigma_1D', 'sigma_3D']>


In [27]:
idxs_halos_dark = np.array([halo_dict['idx_halo_dark'] for halo_dict in featurizer.halo_dicts])

In [28]:
catalog_feature_names = ['M200c', 'c200c', 'a_form']
x_catalog_features = []
for c_feat in catalog_feature_names:
    x_catalog_features.append( f[c_feat][idxs_halos_dark] )

In [18]:
x_catalog_features = np.array(x_catalog_features)

In [19]:
x_catalog_features.shape

(3, 4231400)

In [20]:
featurizer.N_halos

170458